# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [18]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [19]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("WeatherPy_Database.csv")

# Display sample data
vacation_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Garden City,US,42.3256,-83.3310,283.68,99,100,0.00,fog
1,1,Bandarbeyla,SO,9.4942,50.8122,299.53,64,24,10.07,few clouds
2,2,Charters Towers,AU,-20.1000,146.2667,300.11,48,97,6.26,overcast clouds
3,3,Mitsamiouli,KM,-11.3847,43.2844,299.31,83,75,4.12,shower rain
4,4,Aquin,HT,18.2833,-73.4000,295.36,81,5,1.61,clear sky


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [20]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    
    #by adding hover_cols, we can see the info associated with each city, 
    hover_cols = ["Hotel Name", "Country","Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [27]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Trelew"]

vacation_end = vacation_df.loc[vacation_df["City"] == "Trelew"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Viedma"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Cordoba"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Rafaela"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [28]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
frames = [vacation_start, vacation_stop1, vacation_stop2, vacation_stop3,vacation_end]

itinerary_df = pd.concat(frames)


itinerary_df = itinerary_df.rename(columns = {"Unnamed: 0":"City_ID"})


# Display sample data
itinerary_df

itinerary_df = itinerary_df.reset_index()

del itinerary_df["index"]

itinerary_df


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,83,Trelew,AR,-43.2490,-65.3051,293.07,43,3,7.59,clear sky
1,49,Viedma,AR,-40.8135,-62.9967,292.28,53,4,5.23,clear sky
2,97,Cordoba,AR,-31.4135,-64.1811,291.90,73,0,2.06,clear sky
3,70,Rafaela,AR,-31.2503,-61.4867,293.14,39,0,1.34,clear sky
4,83,Trelew,AR,-43.2490,-65.3051,293.07,43,3,7.59,clear sky


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [29]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["City","Lat", "Lng"]].copy()



# Display sample data
waypoints_df

,City,Lat,Lng
0,Trelew,-43.2490,-65.3051
1,Viedma,-40.8135,-62.9967
2,Cordoba,-31.4135,-64.1811
3,Rafaela,-31.2503,-61.4867
4,Trelew,-43.2490,-65.3051


## Use GeoViews to create map that shows the four cities in the itinerary

In [30]:
# Configure the map plot by using the itinerary_df
waypoints_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 500,
    color = "City",
    size = 300,
    alpha = 0.5,
    
    
    #by adding hover_cols, we can see the info associated with each city, 
    hover_cols = ["City", "Country","Current Description"]
)

In [31]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [32]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [33]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints
#params["waypoints"] = "41.89,12.48|43.01,11.17|43.14,12.2|40.21,16.68|41.89,12.48"
# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '83ed1ce238d9427292932ae035fc5eb7',
 'waypoints': '-43.249,-65.3051|-40.8135,-62.9967|-31.4135,-64.1811|-31.2503,-61.4867|-43.249,-65.3051'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [34]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response


{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-65.3051, -43.249], 'original_index': 0},
     {'location': [-62.9967, -40.8135], 'original_index': 1},
     {'location': [-64.1811, -31.4135], 'original_index': 2},
     {'location': [-61.4867, -31.2503], 'original_index': 3},
     {'location': [-65.3051, -43.249], 'original_index': 4}],
    'units': 'metric',
    'distance': 3678894,
    'distance_units': 'meters',
    'time': 132433.133,
    'legs': [{'distance': 490848,
      'time': 17481.906,
      'steps': [{'from_index': 0,
        'to_index': 1,
        'distance': 74,
        'time': 8.994,
        'instruction': {'text': 'Drive north on Sarmiento.'}},
       {'from_index': 1,
        'to_index': 2,
        'distance': 126,
        'time': 20.741,
        'instruction': {'text': 'Turn left onto Abrahan Mathews.'}},
       {'from_index': 2,
        'to_index': 4,
        'distance': 242,
        'time': 27.694,
        'instruc

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [35]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [36]:
# Create and empty list to store the longitude of each step
longitudes = []

# Create and empty list to store the latitude of step
latitudes = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step

for index in legs:
    for coordinates in index:
        longitudes.append(coordinates[0])
        latitudes.append(coordinates[1])


#for coordinates in legs:
#    longitudes.append(coordinates[0])
#    latitudes.append(coordinates[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [37]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitudes

route_df["latitude"] = latitudes

# Display sample data

route_df

,longitude,latitude
0,-65.305958,-43.249269
1,-65.306325,-43.248648
2,-65.307752,-43.249093
3,-65.308337,-43.248116
4,-65.308959,-43.247110
...,...,...
12208,-65.302934,-43.249180
12209,-65.304101,-43.249564
12210,-65.304142,-43.249578
12211,-65.305519,-43.250012


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [38]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).opts(color="red",line_width=4,alpha=0.3)


route_path

:Path   [longitude,latitude]

In [39]:
# Display a composed plot by using the route_map and route_path objects

route_map = waypoints_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 500,
    color = "City",
    size = 300,
    alpha = 0.6,
    
    
    #by adding hover_cols, we can see the info associated with each city, 
    hover_cols = ["City", "Country","Current Description"]
)

composed_map = route_map*route_path

composed_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)
   .Path.I   :Path   [longitude,latitude]